In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Get started with your deployed model on GKE

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fgke_model_ui_deployment_notebook_auto.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/gke_model_ui_deployment_notebook_auto.ipynb">
      <img alt="GitHub logo" src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

# Overview

This notebook will guide you through the initial step of testing your recently
deployed model with text prompts. Depending on your deployed model's inference
setup, the notebook utilizes either Text Generation Inference
[TGI](https://huggingface.co/docs/text-generation-inference/en/index) or
[vLLM](https://developers.googleblog.com/en/inference-with-gemma-using-dataflow-and-vllm/#:~:text=model%20frameworks%20simple.-,What%20is%20vLLM%3F,-vLLM%20is%20an),
two efficient serving frameworks that enhance the performance of your GPU model.
Ready to see your deployed model respond? Run the cells below and start
experimenting with different prompts!

### Prerequisites

Before proceeding with this notebook, ensure you have already deployed a model
using the Google Cloud Console. You can find an overview of AI and Machine
Learning services on
[GKE AI/ML](https://console.cloud.google.com/kubernetes/aiml/overview).

### Objective

Enable prompt-based testing of the AI model deployed on GKE

### GPUs

GPUs let you accelerate specific workloads running on your nodes, such as
machine learning and data processing. GKE provides a range of machine type
options for node configuration, including machine types with NVIDIA H100, L4,
and A100 GPUs.

### Understanding the Inference Frameworks

Your model is running on one of two popular and efficient serving frameworks:
vLLM or Text Generation Inference (TGI). The following sections provide a brief
overview of each to give you context on the underlying technology powering your
model.

#### TGI

TGI is a highly optimized open-source LLM serving framework that can increase
serving throughput on GPUs. TGI includes features such as:

*   Optimized transformer implementation with PagedAttention
*   Continuous batching to improve the overall serving throughput
*   Tensor parallelism and distributed serving on multiple GPUs

To learn more, refer to the
[TGI documentation](https://github.com/huggingface/text-generation-inference/blob/main/README.md)

#### vLLM

vLLM is another fast and easy-to-use library for LLM inference and serving. It's
known for its high throughput and efficiency, and it leverages PagedAttention.
Key features include:

*   PagedAttention: Efficient memory management for handling long sequences and
    dynamic workloads.
*   Continuous batching: Maximizes GPU utilization by batching incoming
    requests.
*   High-throughput serving: Designed for production-level serving with low
    latency.
*   Optimized CUDA kernels.

To learn more, refer to the
[vLLM documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/open-models/vllm/use-vllm)

In [ ]:
# @title # Connect to Google Cloud Project
# @markdown #### Run this cell to configure your Google Cloud environment for Kubernetes (GKE) operations.
# @markdown
# @markdown #### Actions:
# @markdown 1.  **Connects to Project:** Retrieves and sets your Google Cloud project ID.
# @markdown 3.  **Installs `kubectl`:** Installs the Kubernetes command-line tool.

import os

# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Set up gcloud.
! gcloud config set project "$PROJECT_ID"
! gcloud services enable container.googleapis.com

# Add kubectl to the set of available tools.
! mkdir -p /tools/google-cloud-sdk/.install
! gcloud components install kubectl --quiet

In [ ]:
# @title # Chat completion for text-only models { vertical-output: true}
# @markdown You may send prompts to the model server for prediction.
# @markdown
# @markdown * **user_prompt (string):** This is the text prompt you provide to the language model. It's the question or instruction e (e.g., "Explain neural networks").
# @markdown * **temperature (number):** This  parameter controls the randomness of the model's output. It influences how the model selects the next token in the sequence it generates. Typical values range from 0.2 to 1.0.
# @markdown * **max_tokens (number):** This parameter refers to the maximum number of tokens (words or sub-word units) that the model is allowed to generate in its response.
# @markdown

import json
import subprocess

import ipywidgets as widgets
from IPython.display import Markdown, clear_output, display

CLUSTER = ""  # @param {type:"string", isTemplate:true}
REGION = ""  # @param {type:"string", isTemplate:true}
NAMESPACE = ""  # @param {type:"string", isTemplate:true}
DEPLOYMENT = ""  # @param {type:"string", isTemplate:true}
DEPLOYMENT_APP_LABEL = ""  # @param {type:"string", isTemplate:true}

SERVICE = f"{DEPLOYMENT}-service"


def _run_kubectl(cmd):
    """Executes a kubectl command and returns its stdout."""
    result = subprocess.run(cmd, capture_output=True, text=True, check=True, timeout=60)
    return result.stdout.strip()


def fetch_cluster_credential(cluster, region, project_id):
    try:
        # Ensure credentials for the target cluster
        cred_cmd = [
            "gcloud",
            "container",
            "clusters",
            "get-credentials",
            cluster,
            f"--location={region}",
            f"--project={project_id}",
        ]
        _run_kubectl(cred_cmd)
    except Exception as e:
        # Original code prints error and returns empty dict
        print(f"Error fetching cluster credentials: {e}")
        return {}


def get_deployment_pod_name(deployment, namespace, deployment_app_label):
    """Finds the running pod name for a given deployment and namespace."""

    cmd = [
        "kubectl",
        "get",
        "pods",
        "-n",
        namespace,
        "-o",
        "json",
        "-l",
        f"app={deployment_app_label}",
        "--field-selector=status.phase=Running",
    ]
    try:
        pods_json = _run_kubectl(cmd)
        pods = json.loads(pods_json)
        if pods.get("items"):
            return pods["items"][0]["metadata"]["name"]
        print(f"No running pods found for {deployment} in {namespace}.")
        return None
    except (
        subprocess.CalledProcessError,
        json.JSONDecodeError,
        IndexError,
        KeyError,
    ) as e:
        print(f"Error getting pod name for {deployment} in {namespace}: {e}")
        return None


def check_inference_label(pod_name, namespace):
    """Checks if the specified pod has the vLLM inference server label."""

    cmd = ["kubectl", "get", "pod", pod_name, "-n", namespace, "-o", "json"]
    try:
        pod_json = _run_kubectl(cmd)
        labels = json.loads(pod_json).get("metadata", {}).get("labels", {})
        return labels.get("ai.gke.io/inference-server") == "vllm"
    except (subprocess.CalledProcessError, json.JSONDecodeError, KeyError) as e:
        print(f"Error checking labels for pod {pod_name} in {namespace}: {e}")
        return False


def get_service_endpoint(service, namespace):
    """Retrieve the service endpoint of the deployment"""
    endpoint_cmd = [
        "kubectl",
        "get",
        "endpoints",
        service,
        "-n",
        namespace,
    ]
    try:
        endpoint_output = _run_kubectl(endpoint_cmd).splitlines()
        if len(endpoint_output) < 2 or len(endpoint_output[1].split()) < 2:
            print(f"Endpoint data incomplete for {service}.")
            return None
        endpoint = endpoint_output[1].split()[
            1
        ]  # Assumes format: NAME ENDPOINTS AGE -> service ip:port,... age
        return endpoint
    except subprocess.CalledProcessError as e:
        print(f"Error getting endpoints for {service}: {e}")
        return None


def process_response(request, pod_name, pod_endpoint, is_vllm_inference, namespace):
    """Sends a request to the pod and processes the response."""

    json_data_escaped = json.dumps(request).replace("'", "'\\''")
    curl_cmd = (
        f"kubectl exec -n {namespace} -t {pod_name} -- curl -s -X POST"
        f' http://{pod_endpoint}/generate -H "Content-Type: application/json"'
        f" -d '{json_data_escaped}' 2> /dev/null"
    )
    try:
        response_raw = _run_kubectl(["bash", "-c", curl_cmd])
        if not response_raw:
            return f"Error: Empty response from pod {pod_name}."
        first_line = response_raw.splitlines()[0]
        data = json.loads(first_line)

        if is_vllm_inference:  # vLLM format
            predictions = data.get("predictions")
            if isinstance(predictions, (list, tuple)) and predictions:
                return predictions[0]
            return f"Error: Unexpected vLLM format. Raw: {first_line}"
        else:  # TGI format
            generated_text = data.get("generated_text")
            if generated_text is not None:
                return generated_text
            return f"Error: Unexpected TGI format. Raw: {first_line}"
    except Exception as e:
        return f"Unexpected error during response processing: {e}"


# --- Widgets Setup ---
user_prompt_widget = widgets.Textarea(
    value="What is AI?",
    description="User Prompt:",
    layout=widgets.Layout(width="95%", height="100px"),
)

temperature_widget = widgets.FloatSlider(
    value=0.50, min=0.0, max=1.0, step=0.01, description="Temperature:"
)

max_tokens_widget = widgets.IntSlider(
    value=250, min=1, max=2048, step=1, description="Max Tokens:"
)

submit_button = widgets.Button(description="Submit")
output_area_response = widgets.Output()


# --- Submit Button Logic ---
def on_submit_clicked(b):
    """Handles the submit button click event."""
    with output_area_response:
        clear_output()

        fetch_cluster_credential(CLUSTER, REGION, PROJECT_ID)

        # retrieve deployment pod
        pod_name = get_deployment_pod_name(DEPLOYMENT, NAMESPACE, DEPLOYMENT_APP_LABEL)
        if not pod_name:
            display(
                Markdown(f"**Error:** Could not find running pod for `{DEPLOYMENT}`.")
            )
            return

        # build the request message
        is_vllm = check_inference_label(pod_name, NAMESPACE)
        request = {
            "max_tokens": max_tokens_widget.value,
            "temperature": temperature_widget.value,
            "prompt" if is_vllm else "inputs": user_prompt_widget.value,
        }

        # retrieve service endpoint for the deployment
        endpoint = get_service_endpoint(SERVICE, NAMESPACE)
        if not endpoint:
            display(Markdown(f"**Error getting endpoints for `{SERVICE}`:**\n"))
            return

        # prompt test the deployment endpoint
        try:
            response = process_response(request, pod_name, endpoint, is_vllm, NAMESPACE)
            display(Markdown(f"**Response:**\n\n{response}"))
        except Exception as e:
            display(Markdown(f"**Unexpected Error:**\n```\n{e}\n```"))


# --- Display Widgets ---
submit_button.on_click(on_submit_clicked)
display(
    user_prompt_widget,
    temperature_widget,
    max_tokens_widget,
    submit_button,
    output_area_response,
)

# Next Steps: Integrating the GKE Service Endpoint

After successfully deploying a model on Google Kubernetes Engine (GKE) and
verifying it via a notebook, the next step is to integrate it into various
applications. This involves making HTTP requests to the service's endpoint from
your application code.

### Exposing the Service

To make your deployed model accessible to applications, you'll need to expose
its service endpoint. Google Kubernetes Engine offers several ways to do this:

1.  **Ingress:** Configure an Ingress resource to route external HTTP(S) traffic
    to your service. Set up Ingress for either an internal Load Balancer
    (accessible only within your VPC) or an external Load Balancer (accessible
    from the internet).
    [Learn more about GKE Ingress](https://cloud.google.com/kubernetes-engine/docs/concepts/ingress).
2.  **Gateway API:** A more modern and feature-rich API for managing traffic
    routing in Kubernetes. Similar to Ingress, Gateway API allows you to define
    how external and internal traffic should be directed to your services.
    [Explore GKE Gateway API](https://cloud.google.com/kubernetes-engine/docs/concepts/gateway-api).

### Setting Up Autoscaling

Ensure your model serving can handle varying traffic by configuring the
Horizontal Pod Autoscaler (HPA). HPA automatically scales the number of Pods
based on resource utilization or custom metrics, optimizing performance and
cost.
[See how to configure HPA](https://cloud.google.com/kubernetes-engine/docs/how-to/horizontal-pod-autoscaling).

### Setting Up Monitoring

Monitor the health and performance of your deployed model using Google Cloud
Managed Service for Prometheus. Configure your model serving to expose
Prometheus metrics for comprehensive insights.
[Get started with Google Cloud Managed Prometheus](https://cloud.google.com/kubernetes-engine/docs/how-to/configure-automatic-application-monitoring).

### Additional Resources:

*   #### Kubernetes Documentation:

    *   Services:
        https://kubernetes.io/docs/concepts/services-networking/service/

*   #### Google Cloud Documentation:

    *   Google Kubernetes Engine (GKE):
        https://cloud.google.com/kubernetes-engine
    *   Cloud Load Balancing:
        https://cloud.google.com/load-balancing/docs/ingress
    *   Gateway API on GKE:
        https://cloud.google.com/kubernetes-engine/docs/concepts/gateway-api
    *   Learn about GPUs in GKE:
        https://cloud.google.com/kubernetes-engine/docs/concepts/gpus

*   #### Python requests Library:

    *   https://requests.readthedocs.io/en/latest/

*   #### LangChain with Google Integrations:

    *   The Langchain documentation is very useful:
        https://python.langchain.com/docs/integrations/providers/google/